# Route 53 Service Notebook

In [ ]:
import boto3

r53_client = boto3.client('route53', region_name='us-west-2')
sd_client = boto3.client('servicediscovery', region_name='us-west-2')

In [ ]:
cf_client = boto3.client('cloudformation')

In [ ]:
response = cf_client.list_stacks(
    StackStatusFilter=[
        'CREATE_COMPLETE'
    ]
)

print(response)

In [ ]:
response = cf_client.describe_stacks(
    StackName='vpc'
)

print(response)

In [ ]:
outputs = response['Stacks'][0]['Outputs']
vpcId = [x for x in outputs if x['OutputKey'] == 'VpcId'][0]['OutputValue']
print (vpcId)

In [ ]:
# List namespaces
response = sd_client.list_namespaces()
print(response)

In [ ]:
# Create a namespace
response = sd_client.create_private_dns_namespace(
    Name='sector7.internal',
    Description='Sector 7',
    Vpc=vpcId
)

print(response)

In [ ]:
# Extract the namespace id
# List namespaces
response = sd_client.list_namespaces()
namespaces = response['Namespaces']
print([x for x in namespaces if x['Name'] == 'sector7.internal'])
namespaceId = [x for x in namespaces if x['Name'] == 'sector7.internal'][0]['Id']
print(namespaceId)

In [ ]:
# Create a service
response = sd_client.create_service(
    Name='examplesvc',
    Description='an example service',
    DnsConfig={
        'NamespaceId': namespaceId,
        'RoutingPolicy': 'WEIGHTED',
        'DnsRecords': [
            {
                'Type': 'CNAME',
                'TTL': 600
            },
        ]
    }
)

print(response)

In [ ]:
# Extract the service id
serviceId = response['Service']['Id']
print(serviceId)

In [ ]:
# Add an instance
response = sd_client.register_instance(
    ServiceId=serviceId,
    InstanceId='a',
    Attributes={
        'AWS_INSTANCE_CNAME': 'www.example.com'
    }
)

print(response)